In [1]:
import numpy as np
from lsst.sims.photUtils import PhotometricParameters
from lsst.sims.photUtils import Bandpass, Sed, SignalToNoise
import os
from lsst.utils import getPackageDir


In [2]:
def coadd_m5(m5s):
    result = 1.25*np.log10(np.sum(10.**(0.8*m5s)))
    return result

In [36]:
def m52snr_gamma(m, m5, gamma=0.038):
    flux_ratio = 10**(0.4*(m-m5))
    snr = np.sqrt((0.04-gamma) * flux_ratio + gamma * flux_ratio * flux_ratio)
    return 1./snr

In [4]:
def m52snr(m, m5):
    snr = 5.*10.**(-0.4*(m-m5))
    return snr


In [5]:
import lsst.syseng.throughputs as st

In [6]:
defaultDirs = st.setDefaultDirs()

In [7]:
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='pachonModtranAtm_12_aerosol.dat')
hardware, system = st.buildHardwareAndSystem(defaultDirs, addLosses=True, 
                                             atmosphereOverride=None, shiftFilters=None)

In [8]:
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = st.buildHardwareAndSystem(defaultDirs, addLosses=True, 
                                             atmosphereOverride=atmos, shiftFilters=None)

In [45]:
m5info = st.makeM5(hardware, system, X=1.0, exptime=15., nexp=1)
u_1exp = m5info['m5'][0]
m=20.
snr1 = m52snr(m, m5info['m5'][0])
m5info['m5']


u    23.377110
g    24.422912
r    23.969605
i    23.537553
z    22.958897
y    22.066503
Name: m5, dtype: float64

In [10]:
m5info = st.makeM5(hardware, system, X=1.0, exptime=15., nexp=2)
snr2 = m52snr(m, m5info['m5'][0])
m5info['m5']

u    23.764100
g    24.806430
r    24.351669
i    23.919008
z    23.339608
y    22.446767
Name: m5, dtype: float64

In [11]:
coadd_m5(np.array([u_1exp, u_1exp]))

23.753397940691482

In [12]:
snr1, snr2

(112.15384902177257, 160.18063506374244)

In [13]:
snr1*np.sqrt(2)

158.60949435893525

In [14]:
m5info = st.makeM5(hardware, system, X=1.0, exptime=30., nexp=1, readnoise=0, darkcurrent=0)
m5info['m5']


u    24.395532
g    24.985710
r    24.448727
i    23.989584
z    23.387838
y    22.483414
Name: m5, dtype: float64

In [15]:
m5info = st.makeM5(hardware, system, X=1.0, exptime=15., nexp=1, readnoise=0, darkcurrent=0)
u_1exp = m5info['m5'][0]
m5info['m5']


u    23.999740
g    24.600868
r    24.066116
i    23.607777
z    23.006924
y    22.103013
Name: m5, dtype: float64

In [16]:
m5info = st.makeM5(hardware, system, X=1.0, exptime=15., nexp=2, readnoise=0, darkcurrent=0)
m5info['m5']


u    24.395532
g    24.985710
r    24.448727
i    23.989584
z    23.387838
y    22.483414
Name: m5, dtype: float64

In [17]:
coadd_m5(np.array([u_1exp, u_1exp]))

24.376027640833001

In [18]:
## sanity check
m5 = 24
m=20
snr1 = m52snr(m, m5)
# The SNR expected from coadding 2 exposures
snr2 = m52snr(m, coadd_m5(np.array([m5,m5])))

In [19]:
snr1, snr2

(199.05358527674866, 281.50427993736827)

In [20]:
snr1*np.sqrt(2)

281.50427993736741

In [21]:
m5info = st.makeM5(hardware, system, X=1.0, exptime=15., nexp=1)
m5info

,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,0.92,0.80824,22.988641,34.553585,26.897666,0.032339,0.050870,0.491829,0.037396,23.055802,0.622630,0.394941,23.377110,383.984415,23.9,23.4
g,0.87,0.76714,22.256461,201.340544,28.357487,0.124068,0.151018,0.213424,0.038221,24.407022,0.177957,0.091819,24.422912,562.239590,25.0,24.6
r,0.83,0.73426,21.196219,404.863219,28.143236,0.101849,0.114369,0.125876,0.038573,24.432733,0.096511,0.046188,23.969605,700.718410,24.7,24.3
i,0.80,0.70960,20.477858,572.055222,27.829843,0.076313,0.083386,0.096232,0.038721,24.319891,0.070224,0.032234,23.537553,781.641027,24.0,23.6
z,0.78,0.69316,19.599677,856.684655,27.417680,0.052208,0.055617,0.068671,0.038903,24.152838,0.048027,0.020925,22.958897,911.180382,23.3,22.9
y,0.76,0.67672,18.611811,1141.115493,26.639452,0.025494,0.029824,0.170304,0.039012,23.726175,0.036509,0.015203,22.066503,1012.210160,22.1,21.7


In [22]:
m5info = st.makeM5(hardware, system, X=1.0, exptime=15., nexp=2)
m5info

,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,0.92,0.80824,22.988641,69.107171,26.897666,0.032339,0.050870,0.491829,0.038140,23.066504,0.631432,0.405153,23.764100,537.709569,23.9,23.4
g,0.87,0.76714,22.256461,402.681088,28.357487,0.124068,0.151018,0.213424,0.038734,24.414253,0.179280,0.094867,24.806430,789.848688,25.0,24.6
r,0.83,0.73426,21.196219,809.726438,28.143236,0.101849,0.114369,0.125876,0.038986,24.438509,0.097058,0.048254,24.351669,985.707537,24.7,24.3
i,0.80,0.70960,20.477858,1144.110445,27.829843,0.076313,0.083386,0.096232,0.039091,24.325059,0.070576,0.033985,23.919008,1100.157846,24.0,23.6
z,0.78,0.69316,19.599677,1713.369310,27.417680,0.052208,0.055617,0.068671,0.039221,24.157261,0.048230,0.022357,23.339608,1283.364512,23.3,22.9
y,0.76,0.67672,18.611811,2282.230987,26.639452,0.025494,0.029824,0.170304,0.039299,23.730151,0.036647,0.016459,22.446767,1426.248407,22.1,21.7


In [23]:
m5info = st.makeM5(hardware, system, X=1.0, exptime=30., nexp=1)
m5info

,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,0.92,0.80824,22.988641,69.107171,26.897666,0.032339,0.050870,0.491829,0.037729,23.283491,0.414446,0.249742,23.981087,440.304487,23.9,23.4
g,0.87,0.76714,22.256461,402.681088,28.357487,0.124068,0.151018,0.213424,0.038638,24.493159,0.100373,0.050371,24.885337,734.482292,25.0,24.6
r,0.83,0.73426,21.196219,809.726438,28.143236,0.101849,0.114369,0.125876,0.038943,24.483056,0.052512,0.024425,24.396215,946.083535,24.7,24.3
i,0.80,0.70960,20.477858,1144.110445,27.829843,0.076313,0.083386,0.096232,0.039063,24.357882,0.037753,0.016733,23.951831,1067.397139,24.0,23.6
z,0.78,0.69316,19.599677,1713.369310,27.417680,0.052208,0.055617,0.068671,0.039204,24.179940,0.025551,0.010615,23.362287,1256.835447,23.3,22.9
y,0.76,0.67672,18.611811,2282.230987,26.639452,0.025494,0.029824,0.170304,0.039288,23.747481,0.019317,0.007558,22.464097,1403.663766,22.1,21.7


In [24]:
def blah(exptime = 15, nexp=2):
    effarea=np.pi*(6.423/2*100)**2
    X=1.0
    readnoise=8.8
    othernoise=0
    darkcurrent=0.2
    photParams_std = PhotometricParameters(exptime=exptime, nexp=nexp,
                                               gain=1.0, effarea=effarea, readnoise=readnoise,
                                               othernoise=othernoise, darkcurrent=darkcurrent)

    flatSed = Sed()
    flatSed.setFlatSED()
    return flatSed.calcADU(system['u'], photParams=photParams_std)

In [25]:
blah(exptime=15, nexp=2)

1722613205177.5276

In [26]:
blah(exptime=30, nexp=1)

1722613205177.5276

In [27]:
blah(exptime=15, nexp=1)*2


1722613205177.5276

In [28]:
darksky = Sed()
darksky.readSED_flambda(os.path.join(getPackageDir('syseng_throughputs'),
                                     'siteProperties', 'darksky.dat'))
exptime = 15.
effarea=np.pi*(6.423/2*100)**2
nexp=1
X=1.0
readnoise= 0 #8.8
othernoise=0
darkcurrent= 0 #0.2
photParams_std = PhotometricParameters(exptime=exptime, nexp=nexp,
                                           gain=1.0, effarea=effarea, readnoise=readnoise,
                                           othernoise=othernoise, darkcurrent=darkcurrent)


In [29]:
SignalToNoise.calcM5(darksky, system['u'], hardware['u'],photParams_std, FWHMeff=0.7)

24.275559432785247

In [30]:
exptime = 15.
effarea=np.pi*(6.423/2*100)**2
nexp=2
X=1.0
readnoise= 0 #8.8
othernoise=0
darkcurrent= 0 #0.2
photParams_std = PhotometricParameters(exptime=exptime, nexp=nexp,
                                           gain=1.0, effarea=effarea, readnoise=readnoise,
                                           othernoise=othernoise, darkcurrent=darkcurrent)
SignalToNoise.calcM5(darksky, system['u'], hardware['u'],photParams_std, FWHMeff=0.7)

24.677455815778615

In [31]:
exptime = 30.
effarea=np.pi*(6.423/2*100)**2
nexp=1
X=1.0
readnoise= 0 #8.8
othernoise=0
darkcurrent= 0 #0.2
photParams_std = PhotometricParameters(exptime=exptime, nexp=nexp,
                                           gain=1.0, effarea=effarea, readnoise=readnoise,
                                           othernoise=othernoise, darkcurrent=darkcurrent)
SignalToNoise.calcM5(darksky, system['u'], hardware['u'],photParams_std, FWHMeff=0.7)

24.677455815778615

In [32]:
coadd_m5(np.array([24.275559432785244]*2))

24.65184692736522

In [33]:
30.9702389489*np.sqrt(2)

43.798531951469855